In [1]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [2]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Read in helper files

# Enables State Full Name to Abbreviation
state_dict = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/List-of-US-States/master/states.csv', header=None, skiprows=1 , index_col=0, squeeze=True).to_dict()

# Filter to PCE top Level Categores
pce_levels = pd.read_csv('./supplemental_data/pce_categories.csv', header=None)
pce_levels = pce_levels[0].to_list()

Files cleaned in order of perceived importance

# Financial Data

### GDP by State and Year

In [4]:
df = pd.read_csv('./dirty/gdp_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [5]:
df.head(2)

,GeoFips,GeoName,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,United States,3,Current-dollar GDP (millions of current dollars),8577552.00,9062817.00,9631172.00,10250952.00,10581929.00,10929108.00,11456450.00,12217196.00,13039197.00,13815583.00,14474228.00,14769862.00,14478067.00,15048970.00,15599731.00,16253970.00,16843196.00,17550687.00,18206023.00,18695106.00,19479623.00,20527159.00,21372582.00,20893746.00
1,0,United States,4,Compensation (millions of dollars),4713220.00,5075701.00,5409937.00,5854634.00,6046346.00,6143370.00,6362298.00,6729306.00,7077722.00,7491260.00,7889371.00,8068682.00,7767191.00,7932970.00,8234017.00,8575362.00,8843637.00,9259654.00,9709535.00,9977096.00,10436745.00,10969807.00,11459449.00,11580088.00


In [6]:
# reshape
df = pd.melt(df, id_vars=['GeoFips','GeoName','LineCode','Description'])

In [7]:
df.sample(5)

,GeoFips,GeoName,LineCode,Description,variable,value
5816,26000,Michigan,4,Compensation (millions of dollars),2016,273838.50
3707,24000,Maryland,5,Gross operating surplus (millions of dollars),2009,114157.60
6631,8000,Colorado,4,Compensation (millions of dollars),2019,219975.90
5526,28000,Mississippi,4,Compensation (millions of dollars),2015,57703.40
2205,24000,Maryland,3,Current-dollar GDP (millions of current dollars),2004,245124.50


In [8]:
# rename
df = df.rename(columns={'variable':'year', 'value':'dollars'}) 

# Map full state names to Abbreviated form
df['state'] = df['GeoName'].map(state_dict)

In [9]:
# drop uneeded columns
df = df.drop(columns=['GeoFips','GeoName','LineCode'])

# Turn GDP Values into 1:1 dollars -- Multiply by 1 million
df['dollars'] = df['dollars']*1000000

# clean up column names a little bit
df['Description'] = df['Description'].str.replace("(millions of dollars)", "", regex=False)
df['Description'] = df['Description'].str.replace("(millions of current dollars)", "", regex=False)

In [10]:
df.sample(5)

,Description,year,dollars,state
3278,Taxes on production and imports (TOPI),2007,59457700000.00,NaN
5573,Taxes on production and imports (TOPI),2015,32697800000.00,NC
6924,Subsidies,2020,-4941800000.00,AR
2950,Current-dollar GDP,2006,237569300000.00,WI
6119,Subsidies,2017,-1164000000.00,MI


In [11]:
df = df.pivot_table(index=['state','year'], # columns to keep
                    columns='Description', # unstack this column
                    values='dollars', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

Description,state,year,Subsidies,Taxes on production and imports (TOPI),Compensation,Current-dollar GDP,Gross operating surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00
2,AK,1999,-56400000.00,1983100000.00,13127500000.00,24744300000.00,9690100000.00
3,AK,2000,-57500000.00,2515400000.00,13893000000.00,26806600000.00,10455700000.00
4,AK,2001,-112300000.00,2385900000.00,14883300000.00,28494100000.00,11337100000.00


In [12]:
# Strip WhiteSpace
df.columns = df.columns.str.strip()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   state                                   1224 non-null   object 
 1   year                                    1224 non-null   object 
 2   Subsidies                               1224 non-null   float64
 3   Taxes on production and imports (TOPI)  1224 non-null   float64
 4   Compensation                            1224 non-null   float64
 5   Current-dollar GDP                      1224 non-null   float64
 6   Gross operating surplus                 1224 non-null   float64
dtypes: float64(5), object(2)
memory usage: 67.1+ KB


In [14]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [15]:
df.head(5)

Description,state,year,subsidies,taxes_on_production_and_imports_(topi),compensation,current-dollar_gdp,gross_operating_surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00
2,AK,1999,-56400000.00,1983100000.00,13127500000.00,24744300000.00,9690100000.00
3,AK,2000,-57500000.00,2515400000.00,13893000000.00,26806600000.00,10455700000.00
4,AK,2001,-112300000.00,2385900000.00,14883300000.00,28494100000.00,11337100000.00


In [16]:
df = df.rename(columns={'taxes_on_production_and_imports_(topi}':'gdp_production-import_tax',
                        'compensation':'gdp_compensation',
                        'current-dollar_gdp':'gdp_current_dollar',
                        'gross_operating_surplus':'gdp_gross_operating_surplus'})

In [17]:
# including `index=False` removes unwanted description level
df.to_csv('./clean/gdp_state_annual_clean.csv', index=False)

### PCE Total by Function -  State Annual

Will divide by population to get Per Capita PCE

In [18]:
df = pd.read_csv('./dirty/pce_function_state_annual.csv', skipfooter=4, engine='python')
# verified correct head/tail skip against manual excel inspections

In [19]:
df.head(2)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""00000""",United States,,SAPCE4,1,...,Personal consumption expenditures,Millions of current dollars,5536790.00,5877248.00,6283758.00,6767179.00,7073801.00,7348941.00,7740749.00,8231960.00,8769066.00,9277236.00,9746594.00,10050083.00,9891218.00,10260256.00,10698857.00,11047363.00,11363528.00,11847725.00,12263476.00,12693266.00,13239111.00,13913531.00,14428676.00,14047565.00
1,"""00000""",United States,,SAPCE4,2,...,Household consumption expenditures,Millions of current dollars,5431202.00,5753441.00,6145529.00,6609139.00,6894718.00,7150598.00,7535238.00,8025589.00,8558799.00,9038051.00,9497813.00,9762791.00,9602038.00,9965818.00,10387001.00,10705817.00,11010938.00,11482129.00,11891892.00,12291833.00,12821163.00,13468865.00,13988786.00,13526689.00


In [20]:
# drop initially unnecessary columns
df = df.drop(columns=['Region','TableName','LineCode','IndustryClassification'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoFIPS','GeoName','Description','Unit'])

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

# rename columns
df = df.rename(columns={'variable':'year', 'Description':'pce_description','value':'pce_value'})

# strip white space from descriptions
df['pce_description'] = df['pce_description'].str.strip()

# remove foreign travel(negative values cause mathmatical error)
df = df[df['pce_description'].isin(pce_levels)]

# convert pce_value to float for maths
df['pce_value'] = df['pce_value'].astype('float64')

# Convert pce_value to 1:1 dollars
df['pce_value'] = df['pce_value'].multiply(1000000)

# final drop of unnecessary columns
df = df.drop(columns=['GeoName','Unit','GeoFIPS'])

In [21]:
df = df.pivot_table(index=['year','state'], # columns to keep
                    columns='pce_description', # unstack this column
                    values='pce_value', # populate this this column
                    aggfunc='first'
                    ).reset_index()

#cleanup columns names
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

In [22]:
df.head(2)

pce_description,year,state,"clothing,_footwear,_and_related_services",communication,education,financial_services_and_insurance,food_and_beverages_purchased_for_off-premises_consumption,food_services_and_accommodations,"furnishings,_household_equipment,_and_routine_household_maintenance",health,"housing,_utilities,_and_fuels",other_goods_and_services,personal_consumption_expenditures,recreation,transportation
0,1997,AK,552000000.00,193700000.00,255200000.00,848300000.00,1450000000.00,1091000000.00,701300000.00,1965500000.00,2457500000.00,1146800000.00,14205600000.00,1451200000.00,1711600000.00
1,1997,AL,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00


In [23]:
# add PCE to value columns
cols = df.columns[~df.columns.str.contains('year|state')]

df.rename(columns = dict(zip(cols, 'pce_' + cols)), inplace=True)

In [24]:
df.columns = df.columns.str.replace(',','')

df = df.rename(columns={'pce_personal_consumption_expenditures':'pce_total'})

df.head(5)

pce_description,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,1997,AK,552000000.00,193700000.00,255200000.00,848300000.00,1450000000.00,1091000000.00,701300000.00,1965500000.00,2457500000.00,1146800000.00,14205600000.00,1451200000.00,1711600000.00
1,1997,AL,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00
2,1997,AR,2186700000.00,881100000.00,715600000.00,2953100000.00,3952400000.00,2370000000.00,2245300000.00,7505800000.00,7595600000.00,2464000000.00,41941200000.00,3290300000.00,5312800000.00
3,1997,AZ,3830400000.00,1930400000.00,1600000000.00,6674700000.00,8135900000.00,6703500000.00,5342900000.00,13544900000.00,20212800000.00,6564900000.00,96365600000.00,8586500000.00,12454400000.00
4,1997,CA,33257500000.00,15812800000.00,12700500000.00,51494500000.00,54740400000.00,42657600000.00,32307800000.00,106739900000.00,132581500000.00,42047000000.00,676652200000.00,69924000000.00,75773900000.00


In [25]:
df.to_csv('./clean/pce_function_state_annual_clean.csv', index=False)

### Per Capita Personal Income

In [26]:
df = pd.read_csv('./dirty/personal_income_per_capita_state_annual.csv',skiprows=5, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [27]:
df.sample(5)

,GeoFips,GeoName,LineCode,Description,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
67,25000,Massachusetts,2,Population (persons) 1/,4229000,4250000,4248000,4259000,4282000,4305000,4343000,4355000,4358000,4365000,4347000,4318000,4400000,4386000,4277000,4361000,4310000,4536000,4589000,4674000,4741000,4686000.00,4654000.00,4650000.00,4806000.00,4910000.00,4882000.00,4891000.00,4929000.00,5010000.00,5117000.00,5160000.00,5219000.00,5263000.00,5344000.00,5448000.00,5502000.00,5535000.00,5594000.00,5618000.00,5650000.00,5703706.00,5738589.00,5762122.00,5783809.00,5776992.00,5762138.00,5748868.00,5744097.00,5742923.00,5746188.00,5746075.00,5768685.00,5771222.00,5799405.00,5840774.00,5880734.00,5902677.00,5935205.00,5979983.00,6015478.00,6022639.00,6018470.00,6028709.00,6060569.00,6095241.00,6141445.00,6179756.00,6226058.00,6271838.00,6317345.00,6361104.00,6397634.00,6417206.00,6422565.00,6412281.00,6403290.00,6410084.00,6431559.00,6468967.00,6517613.00,6569537.00,6629863.00,6692712.00,6756583.00,6819480.00,6865334.00,6908516.00,6958374.00,6994054.00,7016618.00,7022220.00,6984723.00
119,42000,Pennsylvania,3,Per capita personal income (dollars) 2/,776,717,605,455,423,487,524,608,643,570,609,659,786,965,1169,1272,1305,1305,1378,1437,1403,1552.00,1719.00,1797.00,1903.00,1835.00,1923.00,2071.00,2184.00,2175.00,2247.00,2305.00,2342.00,2452.00,2530.00,2684.00,2854.00,3066.00,3279.00,3546.00,3862.00,4135.00,4362.00,4753.00,5219.00,5743.00,6259.00,6838.00,7505.00,8283.00,9155.00,10034.00,11088.00,11859.00,12433.00,13455.00,14308.00,15031.00,15885.00,17150.00,18543.00,19566.00,20244.00,21250.00,21930.00,22616.00,23583.00,24743.00,25967.00,27336.00,28489.00,30443.00,31634.00,32133.00,33170.00,34984.00,36410.00,38109.00,40337.00,41728.00,40530.00,42251.00,44211.00,45855.00,46213.00,48063.00,49927.00,51181.00,52499.00,55184.00,56742.00,60720.00,64054.00
136,49000,Utah,2,Population (persons) 1/,508000,509000,513000,517000,520000,522000,526000,527000,529000,535000,543000,552000,551000,582000,627000,616000,596000,638000,636000,653000,671000,696000.00,706000.00,724000.00,739000.00,750000.00,783000.00,809000.00,826000.00,845000.00,870000.00,900000.00,936000.00,958000.00,974000.00,978000.00,994000.00,1009000.00,1019000.00,1029000.00,1047000.00,1065672.00,1100733.00,1134601.00,1168784.00,1198793.00,1233935.00,1272365.00,1316421.00,1364235.00,1416094.00,1472595.00,1515472.00,1558314.00,1594943.00,1622342.00,1642910.00,1662833.00,1678120.00,1689372.00,1705865.00,1731223.00,1779780.00,1836799.00,1898404.00,1960446.00,2014177.00,2067976.00,2119784.00,2165960.00,2203482.00,2244502.00,2283715.00,2324815.00,2360137.00,2401580.00,2457719.00,2525507.00,2597746.00,2663029.00,2723421.00,2776094.00,2818261.00,2860480.00,2908068.00,2950652.00,2999092.00,3063043.00,3125784.00,3180896.00,3232701.00,3281684.00,3337975.00
80,29000,Missouri,3,Per capita personal income (dollars) 2/,619,560,489,365,334,368,421,467,508,476,504,520,646,819,980,1083,1147,1200,1230,1378,1332,1438.00,1574.00,1682.00,1763.00,1756.00,1849.00,1937.00,1980.00,2109.00,2196.00,2226.00,2289.00,2407.00,2505.00,2616.00,2828.00,2993.00,3174.00,3492.00,3645.00,3945.00,4222.00,4562.00,5045.00,5386.00,5883.00,6409.00,7070.00,7828.00,8695.00,9370.00,10445.00,11174.00,11885.00,13103.00,13948.00,14616.00,15296.00,16162.00,17277.00,17920.00,18559.00,19578.00,20265.00,21296.00,22236.00,23321.00,24507.00,25540.00,26342.00,27941.00,28690.00,29168.00,30085.00,31510.00,32467.00,34119.00,35502.00,37312.00,36385.00,37118.00,38533.00,40317.00,40569.00,41963.00,43322.00,44410.00,45376.00,47230.00,48922.00,5

In [28]:
# drop un-needed columns
df = df.drop(columns=['LineCode','GeoFips'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoName','Description'])

# rename columns
df = df.rename(columns={'variable':'year'})

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

In [29]:
df.sample(2)

,GeoName,Description,year,value,state
2394,Kentucky,Personal income (millions of dollars),1942,1548,KY
14635,Kentucky,Population (persons) 1/,2010,4349138.00,KY


In [30]:
df.Description.value_counts()

Personal income (millions of dollars)      5580
Population (persons) 1/                    5580
Per capita personal income (dollars) 2/    5580
Name: Description, dtype: int64

In [31]:
# filter to only Per Capita Personal Income
df = df.loc[(df['Description']=='Per capita personal income (dollars) 2/') | (df['Description']=='Population (persons) 1/')]

In [32]:
# Clean up categorical values
df['Description'] = df['Description'].str.replace(' (persons) 1/','', regex=False)
df['Description'] = df['Description'].str.replace(' (dollars) 2/','', regex=False)

In [33]:
# remove nulls generated by region drop from state mapping
df = df.dropna()

df = df.drop(columns='GeoName')

In [34]:
df.head(5)

,Description,year,value,state
4,Population,1929,2644000,AL
5,Per capita personal income,1929,319,AL
10,Population,1929,430000,AZ
11,Per capita personal income,1929,598,AZ
13,Population,1929,1852000,AR


In [35]:
df = df.pivot_table(index=['year','state'], # columns to keep
                    columns='Description', # unstack this column
                    values='value', # populate this this column
                    aggfunc='first'
                    ).reset_index()

In [36]:
df.head(2)

Description,year,state,Per capita personal income,Population
0,1929,AL,319,2644000
1,1929,AR,303,1852000


In [37]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [38]:
df.to_csv('./clean/per_capita_personal_income-population_state_annual_clean.csv', index=False)

### Business Applications State Annual

In [39]:
df = pd.read_csv('./dirty/business_apps_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

# Data exists from 2005-2021

In [40]:
# NAIC Sector Codes
naic_sector_codes = pd.read_csv('./supplemental_data/naic_sectors_codes.csv', skiprows=0 , index_col=0, squeeze=True).to_dict()

# NAIC Series Codes
naic_series_codes = pd.read_csv('./supplemental_data/naic_series_codes.csv', skiprows=0 , index_col=0, squeeze=True).to_dict()

In [41]:
# Filter out incomplete years
df = df[df['year']!=2022]

df = df[df['year']!=2004]

# filter out unwanted series
df = df[~df['series'].isin(['BF_PBF4Q','BF_PBF8Q','BF_SBF4Q','BF_SBF8Q'])]

# filter to only seasonally adjusted data
df = df[df['sa']=='A']

In [42]:
df.tail(5)

,sa,naics_sector,series,geo,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
27275,A,TOTAL,BF_BF8Q,VA,2005,1070,1090,1043,1054,1036,1088,1009,1021,1044,1130,1103,1084
27277,A,TOTAL,BF_BF8Q,WA,2005,1161,1063,1078,1060,993,1109,1162,1156,840,1230,1197,1159
27279,A,TOTAL,BF_BF8Q,WV,2005,D,161,D,148,160,187,167,D,D,201,202,172
27281,A,TOTAL,BF_BF8Q,WI,2005,525,620,609,645,606,624,622,598,467,620,593,553
27283,A,TOTAL,BF_BF8Q,WY,2005,D,D,121,120,119,125,D,D,D,D,D,D


In [43]:
# convert selected columns by index(months) to numeric
df.iloc[:,5:17] = df.iloc[:,5:17].apply(pd.to_numeric, errors='coerce')

# create annual total
df['annual_total'] =  df.iloc[:,5:17].sum(axis=1, min_count=12)

# drop monthly columns after creating annual total
df = df.drop(df.columns[5:17], axis=1)

In [44]:
df.sample(5)

,sa,naics_sector,series,geo,year,annual_total
17225,A,TOTAL,BF_BF8Q,OR,2011,4581.00
23557,A,TOTAL,BA_WBA,HI,2007,3298.00
20435,A,TOTAL,BF_BF4Q,VA,2009,6462.00
23625,A,TOTAL,BA_WBA,VT,2007,1630.00
5502,A,NAICS51,BF_BF4Q,US,2018,5579.00


In [45]:
# Map supplemental NAIC Data to columns
df['naics_sector'] = df['naics_sector'].map(naic_sector_codes)

df['series'] = df['series'].map(naic_series_codes)

In [46]:
df = df.pivot_table(index=['sa','geo','year','naics_sector'], # columns to keep
                    columns='series', # unstack this column
                    values='annual_total', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

series,sa,geo,year,naics_sector,Business Applications,Business Applications from Corporations,Business Formations within Eight Quarters,Business Formations within Four Quarters,High‐Propensity Business Applications BA_WBA – Business Applications with Planned Wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,NaN,1152.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,NaN,1028.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,NaN,953.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,NaN,824.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,NaN,742.00,2122.00


In [47]:
df.columns = df.columns.str.lower().str.replace(' ','_')

df.columns = df.columns.str.replace('-|–','_')

/tmp/ipykernel_752524/3235611047.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('-|–','_')


In [48]:
df.head(5)

series,sa,geo,year,naics_sector,business_applications,business_applications_from_corporations,business_formations_within_eight_quarters,business_formations_within_four_quarters,high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,NaN,1152.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,NaN,1028.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,NaN,953.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,NaN,824.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,NaN,742.00,2122.00


In [49]:
df = df.drop(columns=['business_formations_within_eight_quarters','business_formations_within_four_quarters'])

In [50]:
df.to_csv('./clean/business_apps_state_annual_clean.csv', index=False)

### Poverty Rate

In [51]:
df = pd.read_csv('./dirty/poverty_rate_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [52]:
df.sample(2)

,STATE,Percent,Year,Unnamed: 3
669,Connecticut,8.90,2007,NaN
789,Mississippi,20.10,2005,NaN


In [53]:
# drop unused columns
df = df.drop(columns='Unnamed: 3')

# convert to numeric percent
df['Percent'] = (df['Percent']/100)

# Map states to abbreviated form
df['STATE'] = df['STATE'].map(state_dict)

# convert column names to lower case
df.columns = df.columns.str.lower()

df = df.rename(columns={'percent':'poverty_rate'})

In [54]:
df.sample(2)

,state,poverty_rate,year
1099,NV,0.11,1999
876,FL,0.13,2003


In [55]:
df.to_csv('./clean/poverty_rate_state_annual_clean.csv', index=False)

# Education Data

### Total Spending per Student

In [56]:
df = pd.read_csv('./dirty/tot_spend_student_state_annual.csv', skiprows=6, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [57]:
# Create String
string = 'Total Current Expenditures for Public El-Sec (TE5) per Pupil (MEMBR) [State Finance]'

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

In [58]:
df.sample(1)

,State Name,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,2012-13,2011-12,2010-11,2009-10,2008-09,2007-08,2006-07,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98
50,WYOMING,16228,16134,16513,16431,16047,15903,15815,15988,15815,15232,14628,13856,13266,11437,10190,9308,8985,8645,7835,7425,6842,6218


In [59]:
df['State Name'].unique().shape

(51,)

In [60]:
# reshape dataframe
df = pd.melt(df, id_vars=['State Name'])

#rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_expenditure'})

# trim to desired year digits
df['start_year'] = df['start_year'].str[:5]

# convert state dictionary to uppercase values - match dataframe format
state_dict_upper = {k.upper():v.upper() for k,v in state_dict.items()}

# map state name to state abbreviation
df['state'] = df['state'].map(state_dict_upper)

In [61]:
df = df.rename(columns={'start_year':'year'})

In [62]:
# Check output
df.head(5)

,state,year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831


In [63]:
df.state.unique().shape

(51,)

In [64]:
df.to_csv('./clean/tot_spend_student_state_annual_clean.csv', index=False)

### Total Revenue Per Student

In [65]:
df = pd.read_csv('./dirty/tot_rev_per_student_state_annual.csv', skiprows=0, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [66]:
df.head(5)

,State Name,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2018-19,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2017-18,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2016-17,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2015-16,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2014-15,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2013-14,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2012-13,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2011-12,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2010-11,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2009-10,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2008-09,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2007-08,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2006-07,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2005-06,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2004-05,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2003-04,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2002-03,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2001-02,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2000-01,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1999-00,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1998-99,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1997-98
0,ALABAMA,11415,10792,10590,10200,9992,9913,9653,9534,9776,9667,9708,10356,9548,8555,8028,7349,6971,6956,6503,6523,5975,5535
1,ALASKA,19553,19038,18897,18851,22379,20447,20312,19034,18699,17759,17319,17471,14304,12849,12632,11576,10928,10717,10275,10118,9532,9222
2,ARIZONA,10396,9697,9214,9293,8995,8703,8616,8613,9111,9343,9002,9457,9023,8071,7814,7550,7791,7214,7071,6455,5988,5812
3,ARKANSAS,11733,11592,11388,10939,10762,10478,10391,10939,10939,10738,10072,9758,9362,9031,8712,7542,7243,7112,6250,6054,5772,5697
4,CALIFORNIA,16014,14535,13965,13606,11786,10985,10481,10590,10790,10239,11180,11228,10857,9909,9234,8980,8975,8363,8306,7465,6750,6572


In [67]:
# Create String
string = 'Total Revenues (TR) per Pupil (MEMBR) [State Finance] '

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

# Reshape df
df = pd.melt(df, id_vars=['State Name'])

# Rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_revenue'})

# remap state name to state abbreviations
df['state'] = df['state'].map(state_dict_upper)

# Use Start year instead of year-span
df['start_year'] = df['start_year'].str[:4]

In [68]:
# Sample output
df.sample(10)

,state,start_year,per_pupil_revenue
57,CT,2017,21680
788,MN,2003,10163
125,MN,2016,15133
636,MS,2006,8399
702,RI,2005,13342
660,WV,2006,10780
623,HI,2006,13514
740,MT,2004,8815
433,MO,2010,11069
865,WI,2002,10052


In [69]:
df = df.rename(columns={'start_year':'year'})

In [70]:
df.to_csv('./clean/tot_revenue_student_state_annual_clean.csv', index=False)

### Demographic Data

#### State median Age

In [71]:
df = pd.read_csv('./dirty/median_age_state_annual_dirty.csv')

In [72]:
df.head(2)

,state,year,median_age
0,Alabama,2005,37.40
1,Alabama,2006,37.20


In [73]:
df['state'] = df['state'].map(state_dict)

In [74]:
df.head(2)

,state,year,median_age
0,AL,2005,37.40
1,AL,2006,37.20


In [75]:
# Write to clean folder
df.to_csv('./clean/median_age_state_annual_clean.csv', index=False)

#### Household Occupant Type

In [76]:
df = pd.read_csv('./dirty/household_occupant_type_state_annual_dirty.csv')

In [77]:
df.head(2)

,state,year,Renter occupied,Total,renter_ratio,owner_ratio
0,Alabama,2005,1223506.00,4442558.00,0.28,0.72
1,Alabama,2006,1170202.00,4483875.00,0.26,0.74


In [78]:
# rename columns
df = df.rename(columns={'Renter occupied':'household_renter_occ','Total':'total_households_occ_type','renter_ratio':'household_renter_occ_ratio',
                        'owner_ratio':'household_owner_occ_ratio'})

In [79]:
# remap states
df['state'] = df['state'].map(state_dict)

In [80]:
df.state.nunique()

51

In [81]:
df.head(2)

,state,year,household_renter_occ,total_households_occ_type,household_renter_occ_ratio,household_owner_occ_ratio
0,AL,2005,1223506.00,4442558.00,0.28,0.72
1,AL,2006,1170202.00,4483875.00,0.26,0.74


In [82]:
df = df.drop(columns=['household_renter_occ','total_households_occ_type'])

df.head(2)

,state,year,household_renter_occ_ratio,household_owner_occ_ratio
0,AL,2005,0.28,0.72
1,AL,2006,0.26,0.74


In [83]:
# Write out
df.to_csv('./clean/household_occupants_ratio_state_annual_clean.csv', index=False)

### Household Occupied binary

In [84]:
df = pd.read_csv('./dirty/household_occupancy-binary_state_annual_dirty.csv')

In [85]:
df

,state,year,variable_label,num_houses
0,Alabama,2005,Total,2082140.00
1,Alabama,2006,Total,2110139.00
2,Alabama,2007,Total,2137012.00
3,Alabama,2008,Total,2158692.00
4,Alabama,2009,Total,2182358.00
...,...,...,...,...
3115,Wyoming,2015,Vacant,40532.00
3116,Wyoming,2016,Vacant,47006.00
3117,Wyoming,2017,Vacant,50937.00
3118,Wyoming,2018,Vacant,48363.00


In [86]:
df.state.nunique()

52

In [87]:
# Map state names
df.state = df.state.map(state_dict)

In [88]:
df.state.nunique()

51

In [89]:
df.head(2)

,state,year,variable_label,num_houses
0,AL,2005,Total,2082140.00
1,AL,2006,Total,2110139.00


In [90]:
df = df.pivot_table(index=['state','year'], # columns to keep
                    columns='variable_label', # unstack this column
                    values='num_houses', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

variable_label,state,year,Occupied,Total,Vacant
0,AK,2005,233252.00,274246.00,40994.00
1,AK,2006,229878.00,276590.00,46712.00
2,AK,2007,236421.00,282271.00,45850.00
3,AK,2008,237607.00,283403.00,45796.00
4,AK,2009,236597.00,283895.00,47298.00


In [91]:
df = df.rename(columns={'Occupied':'occupied_households',
                        'Vacant':'vacant_households',
                        'Total':'total_households_occ_bool'})

In [92]:
df.head(2)

variable_label,state,year,occupied_households,total_households_occ_bool,vacant_households
0,AK,2005,233252.00,274246.00,40994.00
1,AK,2006,229878.00,276590.00,46712.00


In [93]:
df['households_occupied_ratio'] = df['occupied_households'].div(df['total_households_occ_bool'])
df.head(2)

variable_label,state,year,occupied_households,total_households_occ_bool,vacant_households,households_occupied_ratio
0,AK,2005,233252.00,274246.00,40994.00,0.85
1,AK,2006,229878.00,276590.00,46712.00,0.83


In [94]:
df = df.drop(columns=['occupied_households','total_households_occ_bool','vacant_households'])

In [95]:
df.to_csv('./clean/household_occupancy_binary_state_annual_clean.csv', index=False)

### Median State Income

In [96]:
df = pd.read_csv('./dirty/median_income_state_annual_dirty.csv')

df.head(2)

,state,year,median_income
0,Alabama,2005,36879.00
1,Alabama,2006,38783.00


In [97]:
# remap states
print('before: ', df.state.nunique())

df['state'] = df['state'].map(state_dict)

before:  52


In [98]:
print('after:', df.state.nunique())

after: 51


In [99]:
df.head(2)

,state,year,median_income
0,AL,2005,36879.00
1,AL,2006,38783.00


In [100]:
# write out
df.to_csv('./clean/median_income_state_annual_clean.csv', index=False)

### Rent Quratiles


In [101]:
df = pd.read_csv('./dirty/rent_quartiles_state_annual_dirty.csv')

In [102]:
df.head(2)

,state,year,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
0,Alabama,2005,272.00,394.00,541.00
1,Alabama,2006,282.00,410.00,562.00


In [103]:
print('before:', df.state.nunique())

df.state = df.state.map(state_dict)

before: 52


In [104]:
print('after:', df.state.nunique())

after: 51


In [105]:
df.head(2)

,state,year,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
0,AL,2005,272.00,394.00,541.00
1,AL,2006,282.00,410.00,562.00


In [106]:
df.to_csv('./clean/rent_quartiles_state_annual_clean.csv', index=False)

### Education Level

In [107]:
df = pd.read_csv('./dirty/edu_level_state_annual_dirty.csv')

In [108]:
df.head(5)

,state,year,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
0,Alabama,2008,210308.00,439587.00,801711.00,170625.00,3078499.00
1,Alabama,2009,218551.00,447402.00,797598.00,168782.00,3115982.00
2,Alabama,2010,225382.00,439863.00,817024.00,182474.00,3161521.00
3,Alabama,2011,236162.00,443488.00,818373.00,190977.00,3193078.00
4,Alabama,2012,238555.00,472037.00,826740.00,193901.00,3209646.00


In [109]:
df['state'] = df['state'].map(state_dict)

In [110]:
df.head(5)

,state,year,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
0,AL,2008,210308.00,439587.00,801711.00,170625.00,3078499.00
1,AL,2009,218551.00,447402.00,797598.00,168782.00,3115982.00
2,AL,2010,225382.00,439863.00,817024.00,182474.00,3161521.00
3,AL,2011,236162.00,443488.00,818373.00,190977.00,3193078.00
4,AL,2012,238555.00,472037.00,826740.00,193901.00,3209646.00


In [111]:
df.to_csv('./clean/edu_level_state_annual_clean.csv', index=False)

### Gini Index

In [112]:
df = pd.read_csv('./dirty/gini_index_state_annual_dirty.csv')

df.head(2)

,state,year,gini_index
0,Alabama,2006,0.47
1,Alabama,2007,0.47


In [113]:
df['state'] = df['state'].map(state_dict)

In [114]:
df.head(2)

,state,year,gini_index
0,AL,2006,0.47
1,AL,2007,0.47


In [115]:
df.to_csv('./clean/gini_index_state_annual_clean.csv', index=False)

### Mean Hours worked

In [116]:
df = pd.read_csv('./dirty/mean_hrs_worked_state_annual_dirty.csv')

In [117]:
df.head(2)

,state,year,mean_usual_hours
0,Alabama,2005,39.80
1,Alabama,2006,39.80


In [118]:
df['state'] = df['state'].map(state_dict)

df.head(2)

,state,year,mean_usual_hours
0,AL,2005,39.80
1,AL,2006,39.80


In [119]:
df.to_csv('./clean/hrs_worked_state_annual_clean.csv', index=False)

### House Value Quartiles

In [120]:
df = pd.read_csv('./dirty/house_value_quartiles_state_annual_dirty.csv')

In [121]:
df.head(2)

,state,year,house_low_value_quartile,house_median_value,house_upper_value_quartile
0,Alabama,2005,59400.00,97500.00,162500.00
1,Alabama,2006,64200.00,107000.00,174900.00


In [122]:
df['state'] = df['state'].map(state_dict)

df.head(2)

,state,year,house_low_value_quartile,house_median_value,house_upper_value_quartile
0,AL,2005,59400.00,97500.00,162500.00
1,AL,2006,64200.00,107000.00,174900.00


In [123]:
df.to_csv('./clean/house_value_quartiles_state_annual_clean.csv', index=False)

### Median Monthly House Costs

In [124]:
df = pd.read_csv('./dirty/median_monthly_house_cost_state_annual_dirty.csv')

In [125]:
df.head(2)

,state,year,median_monthly_house_cost
0,Alabama,2005,624.00
1,Alabama,2006,685.00


In [126]:
df['state'] = df['state'].map(state_dict)

In [127]:
df.head(2)

,state,year,median_monthly_house_cost
0,AL,2005,624.00
1,AL,2006,685.00


In [128]:
df.to_csv('./clean/median_monthly_house_cost_state_annual_clean.csv', index=False)